In [1]:
%reload_ext autoreload
%autoreload 2
os.chdir("..")
import toml
from core.dl_framework.utils import read_config

/usr/local/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%bash 
pwd

/content/Template_Project


In [3]:
config_file = toml.load("configs/default_train_config.toml")
setup_config = read_config(config_file)

In [85]:
from core.dl_framework.learner import Learner
from core.dl_framework.data import get_dataset, Dataset, DataLoader, split_data
bs = setup_config["h_batch_size"]
x_train, y_train, x_test, y_test = get_dataset(setup_config["s_source"], setup_config["s_set"], setup_config["p_tmp_data_path"])
train_ds, test_ds =  Dataset(x_train, y_train), Dataset(x_test, y_test)
train_dl, valid_dl = DataLoader(train_ds, bs), DataLoader(test_ds, bs)

In [87]:
learn = Learner(train_ds, setup_config)
learn.fit(4)
# a = iter(learn.data.train_dl)
# b, c = next(a)
learn.

100%|██████████| 750/750 [00:00<00:00, 223576.97it/s]


In [56]:
import numpy as np
arr = np.array([1, 2])
arr = 2
res = False

print(res and arr)


False


In [23]:
monitor = Monitor_Cb(["val_acc"])
early = EarlyStopping_Cb(monitor="val_loss")
recorder = Recorder_Cb("hi")
cbh = CallbackHandler([monitor, early, recorder])
cbh.Monitor_Cb.learn

AttributeError: 'Monitor_Cb' object has no attribute 'learn'

In [11]:
vars(cbh)

{'cbs': [<core.dl_framework.callbacks.Monitor_Cb at 0x7fdc62e61a20>,
 'Monitor_Cb': <core.dl_framework.callbacks.Monitor_Cb at 0x7fdc62e61a20>,
 'EarlyStopping_Cb': <core.dl_framework.callbacks.EarlyStopping_Cb at 0x7fdc62e61ba8>}